In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('data/comp-merged.csv')
dates = df['date']
df.drop(columns=['date', 'diff'], axis=1, inplace=True)
df.head()

,value_acc,value_met,bias_coef
0,0.000000,4.1,0.000000
1,28.224999,16.0,1.764051
2,44.069999,37.1,1.187867
3,0.920000,5.1,0.180389
4,24.969999,17.0,1.468815


In [3]:
X = df.drop(columns=['value_met'], axis=1)
y = df['value_met']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32, shuffle=False)#, random_state=42)

In [5]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [6]:
svr_params = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

tuned_svr = GridSearchCV(SVR(), svr_params, cv=5, n_jobs=-1, verbose=1)
tuned_svr.fit(X_train, y_train)

print(tuned_svr.best_params_)
print(tuned_svr.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
0.6449257540534313


In [7]:
model = tuned_svr.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

RMSE: 3.5800020178046705
MSE: 12.816414447485512
MAE: 0.9320271764911287


In [8]:
xgb_params = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.3, 0.5]
}

tuned_xgb = GridSearchCV(xgb.XGBRegressor(), xgb_params, cv=5, n_jobs=-1, verbose=1)
tuned_xgb.fit(X_train, y_train)

print(tuned_xgb.best_params_)
print(tuned_xgb.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
{'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 1000}
0.9034978791992863


In [9]:
model = tuned_xgb.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

RMSE: 1.5142441615902171
MSE: 2.2929353809100594
MAE: 0.519724871923


In [10]:
rfr_params = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'random_state': [32]
}
tuned_rfr = GridSearchCV(RandomForestRegressor(), rfr_params, cv=5, n_jobs=-1, verbose=1)
tuned_rfr.fit(X_train, y_train)

print(tuned_rfr.best_params_)
print(tuned_rfr.best_score_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'max_depth': 9, 'min_samples_split': 2, 'n_estimators': 1000, 'random_state': 32}
0.8810183012918638


In [11]:
model = tuned_rfr.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

RMSE: 1.8419998727000533
MSE: 3.3929635310270125
MAE: 0.5414818194976724


In [12]:
model = tuned_xgb.best_estimator_
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
print(f'RMSE test: {RMSE(y_test, y_pred_test)}')
print(f'MSE test: {MSE(y_test, y_pred_test)}') 
print(f'MAE test: {MAE(y_test, y_pred_test)}')
print(f'RMSE train: {RMSE(y_train, y_pred_train)}')
print(f'MSE train: {MSE(y_train, y_pred_train)}')
print(f'MAE train: {MAE(y_train, y_pred_train)}')

RMSE test: 1.5142441615902171
MSE test: 2.2929353809100594
MAE test: 0.519724871923
RMSE train: 1.5452234338830801
MSE train: 2.3877154606214175
MAE train: 0.3921217625524016


In [13]:
y_pred = np.concatenate((y_pred_train, y_pred_test))
y_true = np.concatenate((y_train, y_test))
print(f'RMSE all: {RMSE(y_true, y_pred)}')
print(f'MSE all: {MSE(y_true, y_pred)}')
print(f'MAE all: {MAE(y_true, y_pred)}')

RMSE all: 1.539071784414089
MSE all: 2.368741957579568
MAE all: 0.41766592743563025


In [14]:
pd.DataFrame({'date': dates, 'value': y_pred}).to_csv('data/beirut-daily-corrected.csv', index=False)

In [15]:
pd.DataFrame({'date': dates, 'value_acc_new': y_pred, 'value_met': y_true}).to_csv('data/beirut-daily-comp.csv', index=False)